- add prob to also cut one of adjacent box with 30%

In [1]:
PROB_TO_CUT_ONE_OF_ADJACENT = 0.3
MAX_GRASS_RATIO = 0.25

In [2]:
from dataclasses import dataclass


@dataclass(frozen=True)
class GrassCutterState:
    size: int
    grass_position: tuple[tuple[int, int]]
    current_row: int = 0
    current_col: int = 0
    move_times: int = 0

    def __repr__(self):
        state_array = []
        for row in range(self.size):
            row_array = []
            for col in range(self.size):
                has_grass = (row, col) in self.grass_position
                is_robot = (row, col) == (self.current_row, self.current_col)
                if has_grass and is_robot:
                    row_array.append("R")
                elif has_grass:
                    row_array.append("G")
                elif is_robot:
                    row_array.append("r")
                else:
                    row_array.append(" ")
            state_array.append(row_array)
        return "\n".join(["|".join(row) for row in state_array]) + f"\nMove times: {self.move_times}\n"

    def __hash__(self):
        return hash((self.size, self.grass_position, self.current_row, self.current_col, self.move_times))

    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

    def __lt__(self, other):
        return self.move_times < other.move_times

In [3]:
# Create environment
import random


class NonDeterministicGrassCutterEnvironment:
    move_str_to_delta = {"U": (-1, 0), "D": (1, 0), "L": (0, -1), "R": (0, 1)}

    @classmethod
    def create_init_state(cls, size: int, random_seed: int = 4242) -> GrassCutterState:
        random.seed(random_seed)
        # Random amount of grass with uniform distribution
        amount_of_grass = max(random.uniform(0, MAX_GRASS_RATIO) * size**2, 1)
        # Random position of grass
        grass_position = tuple(
            random.sample([(row, col) for row in range(size) for col in range(size)], int(amount_of_grass))
        )

        return GrassCutterState(size, grass_position)

    @classmethod
    def get_valid_moves(cls, state: GrassCutterState) -> list[str]:
        valid_moves = ["C"]
        if state.current_row > 0:
            valid_moves.append("U")
        if state.current_row < state.size - 1:
            valid_moves.append("D")
        if state.current_col > 0:
            valid_moves.append("L")
        if state.current_col < state.size - 1:
            valid_moves.append("R")
        return valid_moves

    @classmethod
    def get_all_possibly_moved_state_with_prob(
        cls, state: GrassCutterState, move: str
    ) -> list[tuple[float, GrassCutterState]]:
        """
        Return list of (probability, next_state)
        """
        if move not in cls.get_valid_moves(state):
            raise ValueError(f"Invalid move {move}")
        if move == "C":
            # Case 1: Cut only the grass at the current position
            case1_new_grass_position = tuple(
                [pos for pos in state.grass_position if pos != (state.current_row, state.current_col)]
            )
            case1_new_state = GrassCutterState(
                state.size, case1_new_grass_position, state.current_row, state.current_col, state.move_times + 1
            )
            # Case 2: Cut the grass at the current position and one of the adjacent grass
            adjacent_grass = []
            for delta_row, delta_col in cls.move_str_to_delta.values():
                new_row = state.current_row + delta_row
                new_col = state.current_col + delta_col
                if (new_row, new_col) in state.grass_position:
                    adjacent_grass.append((new_row, new_col))
            randomed_adjacent_grass = random.choice(adjacent_grass) if adjacent_grass else None
            case2_new_grass_position = tuple(
                [
                    pos
                    for pos in state.grass_position
                    if pos != (state.current_row, state.current_col) and pos != randomed_adjacent_grass
                ]
            )
            case2_new_state = GrassCutterState(
                state.size, case2_new_grass_position, state.current_row, state.current_col, state.move_times + 1
            )
            return [(1 - PROB_TO_CUT_ONE_OF_ADJACENT, case1_new_state), (PROB_TO_CUT_ONE_OF_ADJACENT, case2_new_state)]
        else:
            delta_row, delta_col = cls.move_str_to_delta[move]
            new_row = state.current_row + delta_row
            new_col = state.current_col + delta_col
            return [(1, GrassCutterState(state.size, state.grass_position, new_row, new_col, state.move_times + 1))]

    @classmethod
    def apply_move(cls, state: GrassCutterState, move: str) -> GrassCutterState:
        all_possibly_moved_state_with_prob = cls.get_all_possibly_moved_state_with_prob(state, move)
        result_state = [element[1] for element in all_possibly_moved_state_with_prob]
        prob_of_state = [element[0] for element in all_possibly_moved_state_with_prob]
        return random.choices(result_state, prob_of_state)[0]

    @classmethod
    def is_terminal(cls, state: GrassCutterState) -> bool:
        return len(state.grass_position) == 0

In [21]:
import time
from concurrent.futures import ThreadPoolExecutor


def run_simulation(
    size: int, total_games: int, search_strategy: callable, use_threads: bool = False, verbose: bool = False, max_workers: int = 4
) -> None:
    max_move_to_end = float("-inf")
    min_move_to_end = float("inf")
    sum_move_to_end = 0
    sum_time_taken = 0
    end_game_count = 0

    def run_simulation_single(i):
        state = NonDeterministicGrassCutterEnvironment.create_init_state(size, random_seed=i)

        start_time = time.time()
        end_state = search_strategy(state)
        end_time = time.time()

        return end_state, end_time - start_time

    if use_threads:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            end_states = list(executor.map(run_simulation_single, range(total_games)))

        for end_state, time_taken in end_states:
            if NonDeterministicGrassCutterEnvironment.is_terminal(end_state):
                end_game_count += 1
                max_move_to_end = max(max_move_to_end, end_state.move_times)
                min_move_to_end = min(min_move_to_end, end_state.move_times)
                sum_move_to_end += end_state.move_times
                sum_time_taken += time_taken

            if verbose:
                print(end_state)
    else:
        for i in range(total_games):
            end_state, time_taken = run_simulation_single(i)

            if NonDeterministicGrassCutterEnvironment.is_terminal(end_state):
                end_game_count += 1
                max_move_to_end = max(max_move_to_end, end_state.move_times)
                min_move_to_end = min(min_move_to_end, end_state.move_times)
                sum_move_to_end += end_state.move_times
                sum_time_taken += time_taken

            if verbose:
                print(end_state)

    print("Random search result:")
    print(f"\t      Total games: {total_games:,}")
    print(f"\t Total end states: {end_game_count:,}({end_game_count/total_games:.2%}%)")
    print(f"\t Max moves to end: {max_move_to_end:,}")
    print(f"\t Min moves to end: {min_move_to_end:,}")
    print(f"\tMean moves to end: {sum_move_to_end / (end_game_count+1e-10):,.2f}")
    print(f"\t Total time taken: {sum_time_taken:,.4f} seconds")
    print(f"\t   Avg time taken: {sum_time_taken / (end_game_count+1e-10):,.4f} seconds")

Use grase amount and move amount quality function
- optimization
  1. will not simulate the same state twice (also count the number of moves)
  2. penalty for the number of moves (to avoid too many moves) by the factor of 2
  3. penalty for the number of left grass (to avoid too many moves) by the factor of 1
  4. as we know the optimal solution is less than 20.
  5. we don't simulate state with quality more than current best quality

In [23]:
# Import priority queue
from queue import PriorityQueue
from functools import partial


def optimized_quality_function(state: GrassCutterState) -> int:
    # Compute inverse grass amount quality function
    inverse_grass_amount = -len(state.grass_position)

    # Compute inverse move times quality function
    inverse_move_times = -state.move_times * 2

    return inverse_grass_amount + inverse_move_times


def quality_based_search(
    state: GrassCutterState, quality_function: callable, verbose: bool = False, max_move: int = float("inf")
) -> GrassCutterState:
    visited_states = set()
    queue = PriorityQueue()
    queue.put((-quality_function(state), state))

    best_state = None
    best_quality = float("-inf")

    while not queue.empty():
        current_quality, current_state = queue.get()
        # Check if the state is already visited
        if current_state in visited_states:
            continue
        visited_states.add(current_state)

        # Check if the current quality is worse than the best quality
        if -current_quality < best_quality:
            continue

        # Compare with the best state if it is terminal
        if NonDeterministicGrassCutterEnvironment.is_terminal(current_state):
            if current_quality > best_quality:
                best_state = current_state
                best_quality = current_quality

        # Try to move to the next state
        for move in NonDeterministicGrassCutterEnvironment.get_valid_moves(current_state):
            # Calculate expected quality of the next state
            expected_quality = 0
            for prob, next_state in NonDeterministicGrassCutterEnvironment.get_all_possibly_moved_state_with_prob(
                current_state, move
            ):
                quality_of_next_state = quality_function(next_state)
                expected_quality += prob * quality_of_next_state
            # Add to the queue
            moved_state = NonDeterministicGrassCutterEnvironment.apply_move(current_state, move)
            queue.put((-expected_quality, moved_state))

    return best_state


run_simulation(
    size=5,
    total_games=5000,
    search_strategy=partial(quality_based_search, quality_function=optimized_quality_function, max_move=20),
    use_threads=True,
    max_workers=256,
    verbose=False,
)

Random search result:
	      Total games: 5,000
	 Total end states: 5,000(100.00%%)
	 Max moves to end: 18
	 Min moves to end: 1
	Mean moves to end: 8.41
	 Total time taken: 273.8078 seconds
	   Avg time taken: 0.0548 seconds
